In [1]:
%run init.ipynb

matchzoo version 1.0
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]
data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`


In [2]:
preprocessor = mz.models.CDSSM.get_default_preprocessor(
    ngram_size = 3
)

In [3]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
valid_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 8404.73it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:04<00:00, 4333.11it/s]
Building Vocabulary from a datapack.: 100%|██████████| 2082963/2082963 [00:00<00:00, 2731408.69it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 8582.66it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:04<00:00, 4417.94it/s]
Processing length_right with len: 100%|██████████| 18841/18841 [00:00<00:00, 815084.44it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 8473.20it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00<

In [4]:
preprocessor.context

{'ngram_process_unit': <matchzoo.preprocessors.units.ngram_letter.NgramLetter at 0x7fed5ef08630>,
 'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x7fed5ef08588>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7fed5ef57160>,
 'vocab_size': 30058,
 'embedding_input_dim': 30058,
 'ngram_vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7fec8ce58080>,
 'ngram_vocab_size': 9654}

In [5]:
triletter_callback = mz.dataloader.callbacks.Ngram(preprocessor, mode='sum')
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=2,
    num_neg=1,
    callbacks=[triletter_callback]
)
testset = mz.dataloader.Dataset(
    data_pack=test_pack_processed,
    callbacks=[triletter_callback]
)

In [6]:
padding_callback = mz.models.CDSSM.get_default_padding_callback(
    with_ngram=True,
    fixed_ngram_length=preprocessor.context['ngram_vocab_size']
)

trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    batch_size=20,
    stage='train',
    sort=False,
    resample=True,
    callback=padding_callback
)
testloader = mz.dataloader.DataLoader(
    dataset=testset,
    batch_size=20,
    stage='dev',
    sort=False,
    callback=padding_callback
)

In [7]:
model = mz.models.CDSSM()

model.params['task'] = ranking_task
model.params['vocab_size'] = preprocessor.context['ngram_vocab_size']
model.params['filters'] = 64
model.params['kernel_size'] = 3
model.params['conv_activation_func'] = 'tanh'
model.params['mlp_num_layers'] = 1
model.params['mlp_num_units'] = 64
model.params['mlp_num_fan_out'] = 64
model.params['mlp_activation_func'] = 'tanh'
model.params['dropout_rate'] = 0.8

model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

CDSSM(
  (net_left): Sequential(
    (0): ConstantPad1d(padding=(0, 2), value=0)
    (1): Conv1d(9654, 64, kernel_size=(3,), stride=(1,))
    (2): Tanh()
    (3): Dropout(p=0.8, inplace=False)
    (4): AdaptiveMaxPool1d(output_size=1)
    (5): Squeeze()
    (6): Sequential(
      (0): Sequential(
        (0): Linear(in_features=64, out_features=64, bias=True)
        (1): Tanh()
      )
      (1): Sequential(
        (0): Linear(in_features=64, out_features=64, bias=True)
        (1): Tanh()
      )
    )
  )
  (net_right): Sequential(
    (0): ConstantPad1d(padding=(0, 2), value=0)
    (1): Conv1d(9654, 64, kernel_size=(3,), stride=(1,))
    (2): Tanh()
    (3): Dropout(p=0.8, inplace=False)
    (4): AdaptiveMaxPool1d(output_size=1)
    (5): Squeeze()
    (6): Sequential(
      (0): Sequential(
        (0): Linear(in_features=64, out_features=64, bias=True)
        (1): Tanh()
      )
      (1): Sequential(
        (0): Linear(in_features=64, out_features=64, bias=True)
        (1): T

In [8]:
optimizer = torch.optim.Adadelta(model.parameters())

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=10
)

In [9]:
trainer.run()

[Iter-102 Loss-0.820]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5302 - normalized_discounted_cumulative_gain@5(0.0): 0.5922 - mean_average_precision(0.0): 0.5508



[Iter-204 Loss-0.636]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5696 - normalized_discounted_cumulative_gain@5(0.0): 0.628 - mean_average_precision(0.0): 0.5867



[Iter-306 Loss-0.500]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5776 - normalized_discounted_cumulative_gain@5(0.0): 0.6344 - mean_average_precision(0.0): 0.5922



[Iter-408 Loss-0.425]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5808 - normalized_discounted_cumulative_gain@5(0.0): 0.6469 - mean_average_precision(0.0): 0.6014



[Iter-510 Loss-0.333]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5925 - normalized_discounted_cumulative_gain@5(0.0): 0.6515 - mean_average_precision(0.0): 0.6139



[Iter-612 Loss-0.307]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5773 - normalized_discounted_cumulative_gain@5(0.0): 0.6491 - mean_average_precision(0.0): 0.6076



[Iter-714 Loss-0.289]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.581 - normalized_discounted_cumulative_gain@5(0.0): 0.6422 - mean_average_precision(0.0): 0.6075



[Iter-816 Loss-0.238]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5709 - normalized_discounted_cumulative_gain@5(0.0): 0.6393 - mean_average_precision(0.0): 0.5938



[Iter-918 Loss-0.220]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5683 - normalized_discounted_cumulative_gain@5(0.0): 0.6377 - mean_average_precision(0.0): 0.5975



[Iter-1020 Loss-0.212]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5781 - normalized_discounted_cumulative_gain@5(0.0): 0.6427 - mean_average_precision(0.0): 0.6004

Cost time: 3756.318562269211s
